In [3]:
# import requests
# import pandas as pd
from tkinter import *
import os
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from contextlib import closing
import threading
from pdf2image import convert_from_path
from PIL import Image, ImageEnhance
import xlsxwriter
import xlrd
headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36"
}
url = "http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&p=page_change&u=%2Fnetahtml%2FPTO%2Fsearch-bool.html&r=0&f=S&l=50&TERM1=search_change&FIELD1=&co1=AND&TERM2=&FIELD2=&d=PTXT"
time_sacle = 30
print("[ run ]---开始爬取数据")
print("[ tip ]---这一步需要的时间，和爬取的时间间隔和爬取的内容数量有关，请耐心等待。")
key_words = ["plant hanger"]
spider_key_words = key_words
for key_word in spider_key_words:
    page_a = []
    page_img = []
    page_href = []
    key_words_url = url
    key_words_url = key_words_url.replace("search_change",str(key_word))
    url_1 = key_words_url
    url_1 = url_1.replace("page_change","1")
    response = requests.get( url_1 , headers=headers )
    if response.status_code == 200:
        print("[一切正常]---开始爬取 "+key_word)
    else:
        print("[出错]---错误状态码："+response.status_code)
    html_doc = response.text
    response.close()
    soup = BeautifulSoup(html_doc, 'html.parser')
    total_len = soup.find("body").find_all("i")[1].find_all("strong")[2].get_text()
    page_num = int(float(total_len)/50.5)+2
    print("一共"+total_len+"个论文，共"+str(page_num-1)+"页")
    soup_tb = soup.find_all("table")
    for tb in soup_tb:
        for tr in tb.find_all("tr"):
            valign_top = list(tr.find_all("td",attrs={"valign":"top"}))
            if len(valign_top)>=2:
                num = valign_top[1].get_text().replace(",","")
                img = "https://pdfpiw.uspto.gov/.piw?Docid="+str(num)
                top = valign_top[2]
                href = "http://patft.uspto.gov/"+str(top.a.get("href"))
                a = top.get_text()
                a = a.replace("\n"," ")
                page_a.append(a)
                page_img.append(img)
                page_href.append(href)
    time.sleep(int(time_sacle))
    if page_num >= 2:
        for i in range(2,page_num):
            url_i = key_words_url
            url_i = url_i.replace("page_change",str(i))
            response = requests.get( url_i , headers=headers )
            if response.status_code == 200:
                print("[一切正常]---爬取到第"+str(i)+"页")
            else:
                print("[出错]---错误状态码："+response.status_code)
            html_doc = response.text
            response.close()
            soup = BeautifulSoup(html_doc, 'html.parser')
            soup_tb = soup.find_all("table")
            for tb in soup_tb:
                for tr in tb.find_all("tr"):
                    valign_top = list(tr.find_all("td",attrs={"valign":"top"}))
                    if len(valign_top)>=2:
                        num = valign_top[1].get_text().replace(",","")
                        img = "https://pdfpiw.uspto.gov/.piw?Docid="+str(num)
                        top = valign_top[2]
                        href = "http://patft.uspto.gov/"+str(top.a.get("href"))
                        a = top.get_text()
                        a = a.replace("\n"," ")
                        page_a.append(a)
                        page_img.append(img)
                        page_href.append(href)
            time.sleep(int(time_sacle))
    page_dict = {"标题":page_a,"专利链接":page_href,"图片链接":page_img}
    page_df = pd.DataFrame(page_dict)
    if not os.path.exists("raw_data"):
        os.mkdir("raw_data")
    page_df.to_excel("./raw_data/"+key_word+".xlsx",index=None)
    print("[ ok ]---"+key_word)
print("[ ok ]---全部导出完毕，保存在raw_data文件夹中")

[ run ]---开始爬取数据
[ tip ]---这一步需要的时间，和爬取的时间间隔和爬取的内容数量有关，请耐心等待。
[一切正常]---开始爬取 plant hanger
一共127个论文，共3页
[一切正常]---爬取到第2页
[一切正常]---爬取到第3页
[ ok ]---plant hanger
[ ok ]---全部导出完毕，保存在raw_data文件夹中
